## Задание по программированию: 1NN против RandomForest

В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами `1NN` с `RandomForestClassifier` из `sklearn` на 1000 деревьях.

In [1]:
from sklearn import datasets, ensemble
from sklearn.metrics import accuracy_score

digits = datasets.load_digits()
X = digits.data
y = digits.target

In [2]:
X_train = X[:X.shape[0]*0.75, :]
X_test  = X[X.shape[0]*0.75:, :]
y_train = y[:X.shape[0]*0.75]
y_test  = y[X.shape[0]*0.75:]

C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
C:\Users\Insassin\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [7]:
def write_answer(data, file_name):
    with open(file_name, 'w') as fout:
        fout.write(str(data))

### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной `N` требует порядка `N log N` сравнений (строже говоря, она работает за `O(N log N))`. Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за `N` сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в `KNeighborsClassifier` из `sklearn` отвечает параметр `algorithm` — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных `ball tree` и `kd tree`.

Доля ошибок, допускаемых `1NN` на тестовой выборке, — ответ в задании 1.

In [3]:
def euclidean_distance(p, q):
    return sum((p - q)**2)


list_of_mins = list()
for i in range(X_test.shape[0]):
    list_of_pairs = list()
    for j in range(X_train.shape[0]):
            list_of_pairs.append((euclidean_distance(X_test[i,:], X_train[j,:]), y_train[j]))
    list_of_mins.append(min(list_of_pairs))

In [9]:
trues, falses = 0.0, 0.0
for i in range(len(list_of_mins)):
    _, m = list_of_mins[i]
    if y_test[i] == m:
        trues  += 1
    else:
        falses +=1

print 'precision is ', (trues / (trues + falses)) * 100.0, '%'
write_answer(falses / (falses + trues), 'ans1.txt')

precision is  96.2222222222 %


### Задание 2

Теперь обучите на обучающей выборке `RandomForestClassifier(n_estimators=1000)` из `sklearn`. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — `1NN`. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [10]:
clf = ensemble.RandomForestClassifier(n_estimators = 1000).fit(X_train, y_train)

pred = clf.predict(X_test)
rm_score = accuracy_score(y_test, pred)
print 'precision is ', rm_score * 100., '%'

write_answer(1 - rm_score, 'ans2.txt')

precision is  93.3333333333 %
